## Bundle

In [1]:
from tg.common import DataBundle, Loc

db = DataBundle.load('files/tsa-test.zip')

In [2]:
db['pymorphy'].head()

,normal_form,alternatives,score,delta_score,POS,animacy,gender,number,case,aspect,transitivity,person,tense,mood,voice,involvement
word_id,,,,,,,,,,,,,,,,
0,—,1,1.000000,1.000000,NONE,None,None,None,None,None,None,None,None,None,None,None
1,фильм,2,0.623076,0.246153,NOUN,inan,masc,sing,nomn,None,None,None,None,None,None,None
2,«,1,1.000000,1.000000,NONE,None,None,None,None,None,None,None,None,None,None,None
3,растворяться,1,1.000000,1.000000,INFN,None,None,None,None,impf,intr,None,None,None,None,None
4,»,1,1.000000,1.000000,NONE,None,None,None,None,None,None,None,None,None,None,None


In [3]:
db['slovnet'].head()

,POS,Animacy,Case,Gender,Number,Aspect,Mood,Person,Tense,VerbForm,Voice,Degree,Variant,Polarity,Foreign,relation,syntax_parent_id
word_id,,,,,,,,,,,,,,,,,
0,PUNCT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,punct,5
1,NOUN,Inan,Nom,Masc,Sing,None,None,None,None,None,None,None,None,None,None,nsubj,5
2,PUNCT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,punct,3
3,NOUN,Inan,Nom,Masc,Plur,None,None,None,None,None,None,None,None,None,None,appos,1
4,PUNCT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,punct,3


- How index is a top-level frame, binding others

In [4]:
from tg.common.ml import batched_training as bt

idb = bt.IndexedDataBundle(
    index_frame = db.index.loc[db.index.split=='train'].sample(5, random_state=43), 
    bundle = db
)
idb.index_frame

,word_id,sentence_id,label,split
sample_id,,,,
58915,58915,58897,0,train
9110,9110,9108,1,train
21253,21253,21248,1,train
687,687,681,0,train
39862,39862,39849,0,train


In [5]:
from tg.grammar_ru import Separator
import numpy as np

df = idb.bundle.src
df = df.loc[df.sentence_id.isin(idb.index_frame.sentence_id)].copy()
df = df.merge(
    idb.index_frame.reset_index().set_index('sentence_id')[['sample_id']], 
    left_on='sentence_id', 
    right_index=True
)
df['mark'] = np.where(df.is_target, df.label.astype(str), 'no')

Separator.Viewer().tooltip('sample_id').highlight('mark', {'1':'#ff0000', '0':'#00ff00'}).to_html_display(df)

## Features

### Extractor

In [6]:
from tg.common.ml import dft

p_extractor = (bt
             .PlainExtractor
             .build('pymorphy')
             .index()
             .join('pymorphy', on_columns='word_id')
             .apply(
                 transformer=dft.DataFrameTransformerFactory.default_factory(),
                 drop_columns=['normal_form', 'alternatives']
             ))
p_extractor.fit_extract(idb)

,score,delta_score,POS_INFN,POS_VERB,animacy_NULL,gender_NULL,number_sing,number_NULL,case_NULL,aspect_impf,aspect_perf,transitivity_intr,person_3per,person_NULL,tense_pres,tense_NULL,mood_indc,mood_NULL,voice_NULL,involvement_NULL
sample_id,,,,,,,,,,,,,,,,,,,,
58915,0.5,0.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
9110,0.5,0.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
21253,0.5,0.5,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
687,-2.0,-2.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
39862,0.5,0.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0


In [7]:
s_extractor = (bt
             .PlainExtractor
             .build('slovnet')
             .index()
             .join('slovnet', on_columns='word_id')
             .apply(
                 transformer=dft.DataFrameTransformerFactory.default_factory(),
                 drop_columns=['syntax_parent_id']
             ))
s_extractor.fit_extract(idb)

,POS_VERB,Animacy_NULL,Case_NULL,Gender_NULL,Number_Sing,Number_NULL,Aspect_Imp,Aspect_Perf,Mood_Ind,Mood_NULL,...,VerbForm_Fin,Voice_Mid,Degree_NULL,Variant_NULL,Polarity_NULL,Foreign_NULL,relation_xcomp,relation_conj,relation_csubj,relation_root
sample_id,,,,,,,,,,,,,,,,,,,,,
58915,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
9110,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
21253,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
687,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
39862,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0


In [8]:
extractor = bt.CombinedExtractor(name = 'features', extractors = [p_extractor, s_extractor])
extractor.fit_extract(idb)

,pymorphy_score,pymorphy_delta_score,pymorphy_POS_INFN,pymorphy_POS_VERB,pymorphy_animacy_NULL,pymorphy_gender_NULL,pymorphy_number_sing,pymorphy_number_NULL,pymorphy_case_NULL,pymorphy_aspect_impf,...,slovnet_VerbForm_Fin,slovnet_Voice_Mid,slovnet_Degree_NULL,slovnet_Variant_NULL,slovnet_Polarity_NULL,slovnet_Foreign_NULL,slovnet_relation_xcomp,slovnet_relation_conj,slovnet_relation_csubj,slovnet_relation_root
sample_id,,,,,,,,,,,,,,,,,,,,,
58915,0.5,0.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
9110,0.5,0.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
21253,0.5,0.5,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
687,-2.0,-2.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
39862,0.5,0.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0


In [9]:
from tg.grammar_ru.components import CoreExtractor

extractor = CoreExtractor(name = 'features')
extractor.fit_extract(idb)

2023-02-03 14:48:43.105881 INFO: Fitting extractor pymorphy in CoreExtractor
2023-02-03 14:48:43.139021 INFO: Success
2023-02-03 14:48:43.139804 INFO: Fitting extractor slovnet_morph in CoreExtractor
2023-02-03 14:48:43.172047 INFO: Success
2023-02-03 14:48:43.174774 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2023-02-03 14:48:43.184256 INFO: Success
2023-02-03 14:48:43.184975 INFO: Fitting extractor syntax_fixes in CoreExtractor
2023-02-03 14:48:43.200043 INFO: Success
2023-02-03 14:48:43.201202 INFO: Fitting extractor syntax_stats in CoreExtractor
2023-02-03 14:48:43.239695 INFO: Success


,pymorphy_score,pymorphy_delta_score,pymorphy_alternatives,pymorphy_POS_INFN,pymorphy_POS_VERB,pymorphy_animacy_NULL,pymorphy_gender_NULL,pymorphy_number_sing,pymorphy_number_NULL,pymorphy_case_NULL,...,slovnet_syntax_relation_root,syntax_fixes_root_No,syntax_fixes_root_Good,syntax_fixes_cycle_status_No,syntax_stats_descendants_relative,syntax_stats_children,syntax_stats_descendants,syntax_stats_sentence_length,syntax_stats_up_depth,syntax_stats_down_depth
sample_id,,,,,,,,,,,,,,,,,,,,,
58915,0.5,0.5,-0.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,-1.106200,-1.306120,-0.935514,0.934281,1.596462,-0.5
9110,0.5,0.5,-0.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.515799,1.793047,0.183136,-1.366653,-1.458211,-0.5
21253,0.5,0.5,-0.5,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,-0.185793,-0.162309,-0.555700,-1.052456,-0.276502,-0.5
687,-2.0,-2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.730449,-0.162309,1.863778,0.899512,0.414753,2.0
39862,0.5,0.5,-0.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,-0.954254,-0.162309,-0.555700,0.585316,-0.276502,-0.5


### Contexts

- What is context features?
- It is not only for NLP, e.g. customer order's history

In [10]:
from tg.grammar_ru.components import PlainContextBuilder

context_builder = PlainContextBuilder(
    include_zero_offset=True,
    left_to_right_contexts_proportion=0.5
)

cdf = context_builder.build_context(idb, 5).sort_index()
cdf

another_word_id
sample_id offset                 
687       -2                  685
          -1                  686
           0                  687
           1                  688
           2                  689
9110      -2                 9108
          -1                 9109
           0                 9110
           1                 9111
           2                 9112
21253     -2                21251
          -1                21252
           0                21253
           1                21254
           2                21255
39862     -2                39860
          -1                39861
           0                39862
           1                39863
           2                39864
58915     -2                58913
          -1                58914
           0                58915
           1                58916
           2                58917

- other contexts are possible, e.g. Slovnet

- now, we need to join with features

In [11]:
tdb = idb.change_index(cdf)
extractor = CoreExtractor(name = 'features', join_column='another_word_id')
extractor.fit_extract(tdb)

2023-02-03 14:48:43.394999 INFO: Fitting extractor pymorphy in CoreExtractor
2023-02-03 14:48:43.446622 INFO: Success
2023-02-03 14:48:43.447274 INFO: Fitting extractor slovnet_morph in CoreExtractor
2023-02-03 14:48:43.477489 INFO: Success
2023-02-03 14:48:43.478352 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2023-02-03 14:48:43.495066 INFO: Success
2023-02-03 14:48:43.495991 INFO: Fitting extractor syntax_fixes in CoreExtractor
2023-02-03 14:48:43.524379 INFO: Success
2023-02-03 14:48:43.525182 INFO: Fitting extractor syntax_stats in CoreExtractor
2023-02-03 14:48:43.565582 INFO: Success


pymorphy_score  pymorphy_delta_score  pymorphy_alternatives  \
sample_id offset                                                                
687       -2            0.512294              0.513019              -0.738792   
          -1            0.512294              0.513019              -0.738792   
           0           -2.011792             -2.457288              -0.244068   
           1            0.508897              0.510280               1.635490   
           2           -1.427516             -1.769718              -0.244068   
9110      -2            0.512294              0.513019              -0.738792   
          -1            0.512294              0.513019              -0.738792   
           0            0.512294              0.513019              -0.738792   
           1            0.512294              0.513019              -0.738792   
           2           -2.516609             -1.863227               0.379210   
21253     -2            0.512294              0.513019              -0.738792   
          -1            0.503526              0.506950               1.798417   
           0            0.512294              0.513019              -0.738792   
           1            0.508897              0.510280               1.635490   
           2           -2.341024             -2.199002               0.106944   
39862     -2            0.512294              0.513019              -0.738792   
          -1            0.508180              0.508180              -0.244068   
           0            0.512294              0.513019              -0.738792   
           1            0.464816              0.482757               1.635490   
           2           -1.476211             -1.594022               0.106944   
58915     -2            0.118704              0.190815               0.601668   
          -1            0.512294              0.513019              -0.738792   
           0            0.512294              0.513019              -0.738792   
           1            0.500305              0.504755               2.436852   
           2            0.512294              0.513019              -0.738792   

                  pymorphy_POS_INFN  pymorphy_POS_PREP  pymorphy_POS_CONJ  \
sample_id offset                                                            
687       -2                    0.0                0.0                0.0   
          -1                    0.0                0.0                0.0   
           0                    1.0                0.0                0.0   
           1                    0.0                1.0                0.0   
           2                    0.0                0.0                0.0   
9110      -2                    0.0                0.0                0.0   
          -1                    0.0                0.0                0.0   
           0                    1.0                0.0                0.0   
           1                    0.0                0.0                0.0   
           2                    0.0                0.0                0.0   
21253     -2                    1.0                0.0                0.0   
          -1                    0.0                0.0                1.0   
           0                    0.0                0.0                0.0   
           1                    0.0                1.0                0.0   
           2                    0.0                0.0                0.0   
39862     -2                    0.0                0.0                0.0   
          -1                    0.0                0.0                1.0   
           0                    1.0                0.0                0.0   
           1                    0.0                1.0                0.0   
           2                    0.0                0.0                0.0   
58915     -2                    0.0                0.0                1.0   
          -1                    0.0                0.0     

But how do we feed it to the network? Need postprocessing

### Assembly Point

In [12]:
from tg.common.ml.batched_training import context as btc

ap_test = btc.ContextualAssemblyPoint(
    name = 'features',
    context_builder = context_builder,
    extractor = bt.PlainExtractor.build('features').index().join('pymorphy','another_word_id').apply(
        transformer=dft.DataFrameTransformerFactory.default_factory(),
        take_columns=['POS']
    ),
    context_length=5
)
ap_test.reduction_type = btc.ReductionType.Pivot
qdf = ap_test.create_extractor().fit_extract(idb)
qdf

,f0a0_POS_ADJF_at_-2,f0a0_POS_ADJF_at_-1,f0a0_POS_ADJF_at_0,f0a0_POS_ADJF_at_1,f0a0_POS_ADJF_at_2,f0a0_POS_ADJS_at_-2,f0a0_POS_ADJS_at_-1,f0a0_POS_ADJS_at_0,f0a0_POS_ADJS_at_1,f0a0_POS_ADJS_at_2,...,f0a0_POS_VERB_at_-1,f0a0_POS_VERB_at_0,f0a0_POS_VERB_at_1,f0a0_POS_VERB_at_2,f0a0_offset_is_presenting_at_-2,f0a0_offset_is_presenting_at_-1,f0a0_offset_is_presenting_at_0,f0a0_offset_is_presenting_at_1,f0a0_offset_is_presenting_at_2,f0a0_present_f0a0
sample_id,,,,,,,,,,,,,,,,,,,,,
58915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1,1,1,1,1,1
9110,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,1,1,1,1,1
21253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1,1,1,1,1,1
687,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,1,1,1,1,1
39862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,1,1,1,1,1


In [13]:
qdf[[c for c in qdf.columns if 'POS_VERB' in c]]

,f0a0_POS_VERB_at_-2,f0a0_POS_VERB_at_-1,f0a0_POS_VERB_at_0,f0a0_POS_VERB_at_1,f0a0_POS_VERB_at_2
sample_id,,,,,
58915,0.0,1.0,0.0,0.0,0.0
9110,0.0,0.0,0.0,0.0,0.0
21253,0.0,0.0,1.0,0.0,0.0
687,0.0,0.0,0.0,0.0,0.0
39862,0.0,0.0,0.0,0.0,0.0


In [14]:
ap_test.reduction_type = btc.ReductionType.Dim3
tn = ap_test.create_extractor().fit_extract(idb)
tn

In [15]:
{k:v for k,v in zip(tn.dim_names, tn.dim_indices)}

{'offset': [-2, -1, 0, 1, 2],
 'sample_id': [58915, 9110, 21253, 687, 39862],
 'features': ['POS_INFN',
  'POS_PREP',
  'POS_CONJ',
  'POS_NOUN',
  'POS_NPRO',
  'POS_ADJF',
  'POS_VERB',
  'POS_ADJS',
  'POS_ADVB',
  'POS_NONE']}

In [16]:
ap_test.reduction_type = btc.ReductionType.Dim3Folded
tn = ap_test.create_extractor().fit_extract(idb)
{k:v for k,v in zip(tn.dim_names, tn.dim_indices)}

{'offset': [-2, -1, 0],
 'sample_id': [58915, 9110, 21253, 687, 39862],
 'features': ['POS_INFN',
  'POS_PREP',
  'POS_CONJ',
  'POS_NOUN',
  'POS_NPRO',
  'POS_ADJF',
  'POS_VERB',
  'POS_ADJS',
  'POS_ADVB',
  'POS_NONE',
  'POS_INFN',
  'POS_PREP',
  'POS_CONJ',
  'POS_NOUN',
  'POS_NPRO',
  'POS_ADJF',
  'POS_VERB',
  'POS_ADJS',
  'POS_ADVB',
  'POS_NONE']}

There are some problems with the current implementation
- requires odd context length and left_to_right_contexts_proportion=0.5
- also naming is imperfect
- needs refactoring

### Batch


In [17]:
ap = btc.ContextualAssemblyPoint(
    name = 'features',
    context_builder = context_builder,
    extractor = CoreExtractor(join_column='another_word_id'),
    context_length=5
)
ap.reduction_type = ap.reduction_type.Dim3Folded
feature_extractor = ap.create_extractor()
feature_extractor.fit_extract(idb)

2023-02-03 14:48:44.163446 INFO: Fitting extractor pymorphy in CoreExtractor
2023-02-03 14:48:44.195553 INFO: Success
2023-02-03 14:48:44.196218 INFO: Fitting extractor slovnet_morph in CoreExtractor
2023-02-03 14:48:44.218326 INFO: Success
2023-02-03 14:48:44.218798 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2023-02-03 14:48:44.226941 INFO: Success
2023-02-03 14:48:44.227739 INFO: Fitting extractor syntax_fixes in CoreExtractor
2023-02-03 14:48:44.247206 INFO: Success
2023-02-03 14:48:44.248702 INFO: Fitting extractor syntax_stats in CoreExtractor
2023-02-03 14:48:44.294044 INFO: Success


In [18]:
from tg.common.ml.batched_training import factories as btf
label_extractor = bt.PlainExtractor.build(btf.Conventions.LabelFrame).index().apply(take_columns='label')
label_extractor.fit_extract(idb)

,label
sample_id,
58915,0
9110,1
21253,1
687,0
39862,0


In [19]:
batcher = bt.Batcher([feature_extractor, label_extractor])
batch = batcher.fit_extract(3, idb)

2023-02-03 14:48:44.607701 INFO: Fitting extractor pymorphy in CoreExtractor
2023-02-03 14:48:44.644598 INFO: Success
2023-02-03 14:48:44.645232 INFO: Fitting extractor slovnet_morph in CoreExtractor
2023-02-03 14:48:44.662010 INFO: Success
2023-02-03 14:48:44.662520 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2023-02-03 14:48:44.670940 INFO: Success
2023-02-03 14:48:44.671459 INFO: Fitting extractor syntax_fixes in CoreExtractor
2023-02-03 14:48:44.678845 INFO: Success
2023-02-03 14:48:44.679303 INFO: Fitting extractor syntax_stats in CoreExtractor
2023-02-03 14:48:44.703634 INFO: Success


In [20]:
batch.index_frame

,word_id,sentence_id,label,split
sample_id,,,,
58915,58915,58897,0,train
9110,9110,9108,1,train
21253,21253,21248,1,train


In [21]:
batch.bundle

{'features': 'AnnotatedTensor', 'label': {'shape': (3, 1), 'index_name': 'sample_id'}}

## Network

- Gorynych network

### Smplest approach

In [22]:
ap.hidden_size = 50
ap.dim_3_network_factory.network_type = btc.Dim3NetworkType.LSTM
head_factory = ap.create_network_factory()
head = head_factory(batch)
head(batch)

tensor([[-0.1500, -0.1214, -0.1068,  0.0030,  0.1218,  0.0272, -0.0503,  0.2663,
         -0.1461, -0.0589, -0.3804,  0.1303, -0.1424,  0.1061, -0.3710, -0.0075,
         -0.0581, -0.1833,  0.0981, -0.0632, -0.0273, -0.0260,  0.0929, -0.0504,
          0.0028,  0.0267, -0.5339,  0.2472,  0.0715, -0.1304, -0.2243, -0.2204,
          0.3329, -0.2702, -0.0710,  0.0485,  0.0111, -0.0410,  0.1033,  0.1871,
          0.1307,  0.0897,  0.2858,  0.1436, -0.3391, -0.0158, -0.1869, -0.2428,
          0.0958, -0.2379],
        [-0.2265,  0.2008,  0.3227,  0.3251,  0.0579, -0.1332,  0.1814,  0.1010,
         -0.3848, -0.1504, -0.0408,  0.1051, -0.3207,  0.2552, -0.1702,  0.1863,
         -0.4109, -0.2061,  0.0869, -0.1804, -0.0610,  0.2455,  0.1471, -0.0021,
          0.0696,  0.0678, -0.6617, -0.1423,  0.2872,  0.3021,  0.0320,  0.0454,
          0.4669, -0.1101, -0.2353, -0.1410, -0.0194, -0.0046, -0.0158, -0.0163,
          0.1797,  0.1610,  0.0623, -0.0873, -0.3102,  0.0974,  0.1517, -0.3117,


In [23]:
head(batch).shape

torch.Size([3, 50])

Why do we need batch here? Because the number of features varies

- this image from OF presentation

In [24]:
import torch

class Network(torch.nn.Module):
    def __init__(self, head, hidden_size):
        super(Network, self).__init__()
        self.head = head
        self.tail = btf.Perceptron(hidden_size, 1)
        
    def forward(self, batch):
        return self.tail(self.head(batch))
    
network = Network(head, ap.hidden_size)
network(batch)

tensor([[0.5633],
        [0.5493],
        [0.5586]], grad_fn=<SigmoidBackward0>)

### Advanced approach

In [25]:

class Network(torch.nn.Module):
    def __init__(self, head, hidden_size):
        super(Network, self).__init__()
        self.head = head
        self.tail = btf.Perceptron(hidden_size, 1)
        
    def forward(self, batch):
        return self.tail(self.head(batch))
    
    class Factory:
        def __init__(self, head_factory):
            self.head_factory = head_factory
            
        def __call__(self, batch):
            head = self.head_factory(batch)
            hidden_size = head(batch).shape[1]
            return Network(head, hidden_size)
            
            
        
    
network = Network(head, ap.hidden_size)
network(batch)

tensor([[0.4969],
        [0.4550],
        [0.5108]], grad_fn=<SigmoidBackward0>)

In [26]:

class Network(torch.nn.Module):
    def __init__(self, head, hidden_size):
        super(Network, self).__init__()
        self.head = head
        self.tail = btf.Perceptron(hidden_size, 1)
        
    def forward(self, batch):
        return self.tail(self.head(batch))
    
    class Factory:
        def __init__(self, head_factory):
            self.head_factory = head_factory
            
        def __call__(self, batch):
            head = self.head_factory(batch)
            intermediate_tensor = head(batch)
            return Network(head, intermediate_tensor)
            
            
        
    
network = Network(head, ap.hidden_size)
network(batch)

tensor([[0.5283],
        [0.5200],
        [0.5392]], grad_fn=<SigmoidBackward0>)

In [27]:
from functools import partial

factory = btf.FeedForwardNetwork.Factory(
    ap.create_network_factory(),
    partial(btf.Perceptron, output_size = 1)
)
network = factory(batch)
network(batch)

tensor([[0.5002],
        [0.5270],
        [0.5789]], grad_fn=<SigmoidBackward0>)

In [28]:
## Training task

In [30]:
from sklearn.metrics import roc_auc_score

class TrainingTask(btf.TorchTrainingTask):
    def __init__(self):
        super(TrainingTask, self).__init__()
        self.metric_pool = bt.MetricPool().add_sklearn(roc_auc_score)
    
    def initialize_task(self, idb):
        self.setup_batcher(idb, [feature_extractor, label_extractor])
        self.setup_model(factory, ignore_consistancy_check=True)
        
task = TrainingTask()
task.settings.epoch_count=10
result = task.run(db)

2023-02-03 14:49:06.499566 INFO: Training starts. Info: {}
2023-02-03 14:49:06.502372 INFO: Ensuring/loading bundle. Bundle before:
{'index': {'shape': (2144, 4), 'index_name': 'sample_id'}, 'slovnet': {'shape': (49990, 17), 'index_name': 'word_id'}, 'syntax_closure': {'shape': (128382, 4), 'index_name': 'entry_id'}, 'syntax_fixes': {'shape': (49990, 4), 'index_name': 'word_id'}, 'syntax_stats': {'shape': (49990, 6), 'index_name': 'word_id'}, 'pymorphy': {'shape': (49990, 16), 'index_name': 'word_id'}, 'src': {'shape': (49990, 17), 'index_name': None}}
2023-02-03 14:49:06.505141 INFO: Bundle loaded
{'index': {'shape': (2144, 4), 'index_name': 'sample_id', 'columns': ['word_id', 'sentence_id', 'label', 'split'], 'index': [3, 21, 39, 45, 76, '...']}, 'slovnet': {'shape': (49990, 17), 'index_name': 'word_id', 'columns': ['POS', 'Animacy', 'Case', 'Gender', 'Number', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'syntax_closure': {'shape': (128382, 4), 'index_name': 'entry_id', 'columns': ['wo

2023-02-03 14:49:17.001300 INFO: ###loss:0.22087117470800877
2023-02-03 14:49:17.001781 INFO: ###iteration:7
2023-02-03 14:49:17.002890 INFO: Epoch 8 of 10
2023-02-03 14:49:17.389859 INFO: Training: 0/1 batch, 0/4 mini-epoch
2023-02-03 14:49:17.444773 INFO: Training: 0/1 batch, 1/4 mini-epoch
2023-02-03 14:49:17.496974 INFO: Training: 0/1 batch, 2/4 mini-epoch
2023-02-03 14:49:17.544744 INFO: Training: 0/1 batch, 3/4 mini-epoch
2023-02-03 14:49:17.594935 INFO: test: 0/1
2023-02-03 14:49:17.791561 INFO: display: 0/1
2023-02-03 14:49:18.006834 INFO: ###roc_auc_score_test:0.7928727932588823
2023-02-03 14:49:18.008535 INFO: ###roc_auc_score_display:0.8140169550574846
2023-02-03 14:49:18.011244 INFO: ###loss:0.21390449674800038
2023-02-03 14:49:18.013939 INFO: ###iteration:8
2023-02-03 14:49:18.015194 INFO: Epoch 9 of 10
2023-02-03 14:49:18.361454 INFO: Training: 0/1 batch, 0/4 mini-epoch
2023-02-03 14:49:18.415685 INFO: Training: 0/1 batch, 1/4 mini-epoch
2023-02-03 14:49:18.485337 INFO: T